In [1]:
import pandas as pd
from sklearn.preprocessing import normalize

# ===== 1. 데이터 불러오기 =====
job_df = pd.read_excel("C:/Users/user/Desktop/비타민/공모전/직업별_하이브리드_만족도정규화_성격프로파일_최종.xlsx")
mbti_df = pd.read_excel("C:/Users/user/Desktop/비타민/공모전/MBTI별_직업_정리_완전버전.xlsx")

# ===== 2. L2 정규화 =====
trait_columns = job_df.columns[1:]
trait_matrix = job_df[trait_columns].values
normalized_matrix = normalize(trait_matrix, axis=1)
df_l2_normalized = pd.DataFrame(normalized_matrix, columns=trait_columns)
df_l2_normalized.insert(0, '직업명', job_df.iloc[:, 0])

# ===== 3. 사용자 MBTI 입력 =====
def get_mbti_input():
    mbti = input("당신의 MBTI를 입력하세요 (예: ENFP): ").strip().upper()
    return mbti

# ===== 4. 사용자 성격 점수 입력 (0~2) =====
def get_user_input(trait_columns):
    print("\n각 성격 항목에 대해 0~2 사이의 점수를 입력하세요. (엔터 시 기본값 1)")
    user_input = {}
    for trait in trait_columns:
        try:
            val = input(f"{trait}: ")
            val = int(val) if val.strip() != "" else 1
            if val < 0 or val > 2:
                print("⚠️ 0~2 사이의 값만 허용됩니다. 기본값 1이 적용됩니다.")
                val = 1
        except:
            print("⚠️ 숫자로 입력해주세요. 기본값 1이 적용됩니다.")
            val = 1
        user_input[trait] = val
    return pd.Series(user_input)

# ===== 5. 추천 직업 계산 =====
def recommend_jobs_by_mbti(mbti_input, user_traits, mbti_df, job_df):
    mbti_jobs = mbti_df.loc[mbti_df['MBTI'] == mbti_input, '직업']
    if mbti_jobs.empty:
        print(f"\n❌ 입력한 MBTI({mbti_input})에 해당하는 추천 직업 정보가 없습니다.")
        return pd.DataFrame({'직업명': [], '예상직업만족도': []})

    job_list = [job.strip() for job in mbti_jobs.values[0].split(',')]
    filtered = job_df[job_df['직업명'].isin(job_list)].copy()

    if filtered.empty:
        print(f"\n⚠️ 추천된 직업 중 성격 가중치 데이터셋과 정확히 일치하는 직업이 없습니다.")
        return pd.DataFrame({'직업명': [], '예상직업만족도': []})

    scores = filtered[trait_columns].multiply(user_traits, axis=1).sum(axis=1)
    results = pd.DataFrame({
        '직업명': filtered['직업명'],
        '예상직업만족도': scores
    }).sort_values(by='예상직업만족도', ascending=False).reset_index(drop=True)

    return results

# ===== 6. 실행 =====
mbti_input = get_mbti_input()
user_traits = get_user_input(trait_columns)
top_jobs = recommend_jobs_by_mbti(mbti_input, user_traits, mbti_df, df_l2_normalized)

# ===== 7. 결과 출력 =====
print("\n📌 당신에게 추천되는 직업 목록 (예상 직업만족도 순):")
print(top_jobs if not top_jobs.empty else "추천 가능한 직업이 없습니다.")


당신의 MBTI를 입력하세요 (예: ENFP):  ESFJ



각 성격 항목에 대해 0~2 사이의 점수를 입력하세요. (엔터 시 기본값 1)


<성격> 성취/노력:  1
<성격> 인내:  1
<성격> 책임성과 진취성:  1
<성격> 리더십:  
<성격> 협조:  1
<성격> 타인에대한 배려:  1
<성격> 사회성:  1
<성격> 자기통제:  1
<성격> 스트레스감내성:  
<성격> 적응성/융통성:  1
<성격> 신뢰성:  1
<성격> 꼼꼼함:  1
<성격> 정직성:  1
<성격> 독립성:  1
<성격> 혁신:  1
<성격> 분석적 사고:  1



📌 당신에게 추천되는 직업 목록 (예상 직업만족도 순):
       직업명   예상직업만족도
0   유치원 교사  0.630148
1  초등학교 교사  0.049423
2      간호사 -0.539883
3    사회복지사 -0.717849
4       비서 -1.644114
